# "fastcore Learnings"

> "Useful fastcore utils"

- toc: true
- branch: master
- badges: false
- comments: true
- categories: [fastai]
- hide: false
- search_exclude: false
- image: images/post-thumbnails/ml.png
- metadata_key1: 
- metadata_key2:

The purpose is to learn few snippets from FASTCORE so that we can improve our daily python skills and understand fastai better


In [1]:
from fastcore.all import * 
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.nb_imports import *

# Store attributes of a class easily

In [5]:

## normal

class something:
    def __init__(self,a,b,c):
        self.a = a
        self.b = b
        self.c = c
    
x = something(2,3,4)

print(x.a)
print(x.b)
print(x.c)


2
3
4


In [3]:
## using fastcore

class something:
    def __init__(self,a,b,c):
        store_attr()     #<--------
    
x = something(2,3,4)

print(x.a)
print(x.b)
print(x.c)

2
3
4


# Avoid boiler plate during Inheritance

In [8]:

## normal

class parent:
    def __init__(self):
         self.message = "I am from parent class"
            
class child(parent):
    def __init__(self, e,f,g):
        store_attr()
        super().__init__()   #<--- need this to init parent class
    
cc = child(2,3,4)


## using fastcore


# Creating a wrapper to the parent. call it BetterParent :) "


class BetterParent(parent, metaclass=PrePostInitMeta):  #<--- inherits both Parent + metaclass = PrePostInitMeta
    def __pre_init__(self, *args, **kargs):  
        
          # PrePostInitMeta has "pre", "post" init methods. 
          # It creates the "Parent" everytime it is invoked by any child
        super().__init__() 
        
class child2(BetterParent):
    def __init__(self): pass  #<--- Just child init. BetterParent runs pre-init and creates parent.
    
class child3(BetterParent):
    def __init__(self): pass  #<--- Just child init. no parent class init ever again!
    
cc2 = child2()


cc3 = child3()

print(cc3.message)

# use cases :  1 to many relationships can be easily creted with much boiler plate with this code


I am from parent class


# Alternative to lambda

In [9]:

import numpy as np

arr = np.array([1,2,3,4,5])

y = lambda x : x.sum() 

y(arr)


15

In [10]:
import numpy as np

arr = np.array([1,2,3,4,5])

y = Self.sum()  #<--- notice capital "S" (instead of s). Avoids lambda x: x

y(arr)

15

# Transforms

In [98]:
class A(Transform):                      #<---- pass the class
    def encodes(self, x): return x+100   #<---- implements the encode func
    def decodes(self, x) : return x-100  #<---- implements the decode func


f=A()
f(1)

101

In [99]:
f.decode(1)  # without the "s"

-99

In [107]:
 #convert any func into a transform using the decorator. adds implementation to the encodes func. 
@Transform
def specialfunction(x): return x+100

specialfunction(100)   # same as  specialfunction.encode(100)
specialfunction.decode(100)   # decode not implemented. just returns input

100

# Pipeline

Executing transforms in a certain order.  It can take a normal function and make it transform internally. see below

In [100]:
def add2(x) : return x+2

def mul100(y): return y*100

pipe = Pipeline([add2, mul100])

pipe(1)  #. (1+2) * 300 = 300

300

In [105]:
# create a transform, add to pipeline

class addsomemore(Transform):
    order =-1   #<---- decreased the order by 1. in the below examples it shift from 3rd to 2nd position
    def encodes(self,x): return x +1000
    def decodes(self,y): return y -1000

adds = addsomemore()

pipe = Pipeline([add2, mul100, adds])

pipe(1)  #. (1+2) * 300 + 1000 = 1300!

##uncomment "order" and run.  Result : (1+2) |||| + 1000 = 1003||| * 1000 = 100300


100300

# Better Partial functions

In [91]:

# normal

def func100(x, y):
    '''this is func100'''
    return x + y

func100.__doc__


'this is func100'

In [106]:
add2sonly = partial(func100, 2)

add2sonly(4)  # 6
add2sonly(10) # 12

add2sonly.__doc__

#lost the doc string :( 

'partial(func, *args, **keywords) - new function with partial application\n    of the given arguments and keywords.\n'

In [94]:
# fastcore

def func100(x, y):
    '''this is func100'''
    return x + y

add2sonly = partialler(func100, 2)  # partialler  (and not partial)

add2sonly(4)  # 6
add2sonly(10) # 12

add2sonly.__doc__  #<----- still has the doc string :)


'this is func100'

# Monkey Patch-ing

In [64]:

##normal

class A: 
    def somefunc(self):
            print("hello this is somefunc")

def mk_patch_func(self):
           print("monkey patch")   
        
A.somfunc = mk_patch_func   # replace the add of somfunc with mk_patch_func
obj = A()
obj.somfunc()  #<---- calls monkey patch function (and not somefunc)

monkey patch


In [69]:
## fastcore


class B: 
    def funcb(self):
            print("hello this is somefunc")

            
@patch                        #<---- decorator
def funcb(self:B):            #<---- pass the class name
           print("monkey patch")   
        

obj1 = B()
obj1.funcb()  #<---- calls monkey patch function (and not somefunc)


monkey patch


## Type Dispatch

By def, When you have multiple implementation of the same function name, depending on the input, appropriate func is called. fastcore enables it for python


In [147]:

@typedispatch                        #<--- add this decorator
def callme(x, y:int):  return x+y

@typedispatch                       #<--- add this decorator
def callme(x, y:str): return f"{x} {y}"


callme(1,2)


3

In [148]:
callme("hello", "world")

'hello world'

References

[Blog post](https://www.kdnuggets.com/2020/10/fastcore-underrated-python-library.html)

[fastcore lib](https://fastcore.fast.ai)